In [1]:
import pandas as pd
import numpy as np

import joblib
import lightgbm
from sklearn.metrics import ndcg_score

In [2]:
df = pd.read_csv('data/filled_training_set1.csv')
# remain_features = ['prop_starrating', 'prop_review_score', 'prop_location_score2', 'promotion_flag']+['srch_id', 'prop_id'] + ['score']
# remain_features = ['prop_starrating', 'prop_review_score', 'prop_location_score2', 'promotion_flag']+['srch_id', 'score'] + ['prop_location_score1', 'prop_brand_bool', 'price_usd', 'prop_country_id','srch_query_affinity_score','visitor_hist_starrating', 'srch_room_count'] + ['comp_avg_rate_percent_diff']
# train_set = train_set[remain_features]
# numeric_cols = ['prop_starrating', 'prop_review_score', 'prop_location_score2','prop_brand_bool','prop_log_historical_price','visitor_hist_starrating', 'visitor_hist_adr_usd', 'comp_avg_rate', 'comp_avg_inv', 'comp_avg_rate_percent_diff']
# data_filled=data_filled
# combined_df = pd.concat([data_filled, data_filled1], ignore_index=True)
# grouped_df = combined_df.groupby('prop_id')[numeric_cols].agg(['mean', 'std', 'median'])
# grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
# combined_df = combined_df.merge(grouped_df, on='prop_id', how='left')
# train_df = combined_df.iloc[:len(data_filled), :]
# test_df = combined_df.iloc[len(data_filled):, :]

# train_set = train_set.drop(columns=['prop_id', 'site_id', 'random_bool', 'srch_destination_id', 'visitor_location_country_id'])


In [3]:
# features = ['prop_starrating', 'prop_review_score', 'prop_brand_bool',
#        'prop_location_score2', 'promotion_flag', 'srch_length_of_stay',
#        'srch_booking_window', 'srch_adults_count', 'srch_children_count',
#        'srch_room_count', 'srch_saturday_night_bool',
#        'srch_query_affinity_score', 'random_bool', 'count_window',
#        'comp_avg_rate', 'comp_avg_inv', 'score',
#        'visitor_hist_starrating_median', 'visitor_hist_adr_usd_median',
#        'lr_score'] + ['srch_id']
train_set = df

train_size = int(len(train_set) * 0.8)
train_df = train_set[:train_size]
validation_df  = train_set[train_size:]

In [4]:
qids_train = train_df.groupby("srch_id")["srch_id"].count().to_numpy()
X_train = train_df.drop(["srch_id", "score"], axis=1)
y_train = train_df["score"]

qids_validation = validation_df.groupby("srch_id")["srch_id"].count().to_numpy()
X_validation = validation_df.drop(["srch_id", "score"], axis=1)
y_validation = validation_df["score"]
print(qids_train)
print(qids_validation)

[28 32  5 ... 33 33  6]
[27 13 32 ... 24 28  6]


In [5]:
model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    verbosity=0,
    num_leaves=64,
    learning_rate=0.1,
    max_depth=-1,
    n_estimators=1000, 
    force_col_wise=True
)
k = 5
# model = lightgbm.LGBMRanker(
#     objective="lambdarank",
#     metric="ndcg",
#     verbosity=0,  # 设置verbosity参数来控制模型的详细程度
#     num_leaves=64,  # 增加树的深度
#     learning_rate=0.2,  # 增加学习率
#     max_depth=-1,  # 树的最大深度，-1 表示不限制
#     # min_child_samples=50,  # 叶子节点的最小样本数
#     # 'subsample': 0.8,  # 数据子抽样率
#     # 'colsample_bytree': 0.8,  # 特征子抽样率
#     # lambda_l1=0.0,  # L1 正则化参数
#     # lambda_l2=0.0,  # L2 正则化参数
#     # 'max_depth': -1,  # 不限制树的深度
#     # 'min_data_in_leaf': 50,  # 设置叶子节点的最小数据量
#     # 'lambda_l1': 0.1,  # 增加L1正则化项
#     # 'lambda_l2': 0.1,  # 增加L2正则化项
#     # 'feature_fraction': 0.8,  # 设置特征抽样比例
#     # 'bagging_fraction': 0.8,  # 设置数据抽样比例
#     # 'bagging_freq': 5,  # 每5次迭代进行一次数据抽样
# )

In [6]:

model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_validation, y_validation)],
    eval_group=[qids_validation],
    eval_at=k,
)

LGBMRanker(force_col_wise=True, metric='ndcg', n_estimators=1000, num_leaves=64,
           objective='lambdarank', verbosity=0)

In [7]:
ndcg_results = model.evals_result_['valid_0']['ndcg@'+str(k)]
print(np.mean(ndcg_results))

0.38166920975915003


In [8]:
output = model.predict(X_train)
print(output)
# ndcg = ndcg_score([y_train], [output])
# 
# print("NDCG:", ndcg)

[-0.41756318 -1.05269154 -1.30460498 ... -1.62165299 -0.4690591
 -1.75923018]


In [9]:
# ndcg_results = []
# for i in range(len(y_grouped)):
#     ndcg = ndcg_score([y_grouped[i]], [output_grouped[i]], k=5)
#     ndcg_results.append(ndcg)
#     if i % 10000 == 0:
#         print(f'Group {i}, ndcg:{ndcg}.')
# # print(ndcg_results)
# print(np.mean(ndcg_results))

In [9]:
qids = train_df.groupby("srch_id")["srch_id"].count().to_numpy()
X = train_df.drop(["srch_id", "score"], axis=1)
y = train_df["score"]
print(qids)
model.fit(
    X=X,
    y=y,
    group=qids,
    eval_at=5,
)

[28 32  5 ... 33 33  6]


LGBMRanker(force_col_wise=True, metric='ndcg', n_estimators=1000, num_leaves=64,
           objective='lambdarank', verbosity=0)

In [12]:
print(X.columns)
print(len(X.columns))

Index(['site_id', 'visitor_location_country_id', 'visitor_hist_starrating',
       'visitor_hist_adr_usd', 'prop_country_id', 'prop_id', 'prop_starrating',
       'prop_review_score', 'prop_brand_bool', 'prop_location_score1',
       'prop_location_score2', 'prop_log_historical_price', 'price_usd',
       'promotion_flag', 'srch_destination_id', 'srch_length_of_stay',
       'srch_booking_window', 'srch_adults_count', 'srch_children_count',
       'srch_room_count', 'srch_saturday_night_bool',
       'srch_query_affinity_score', 'orig_destination_distance', 'random_bool',
       'days', 'ump', 'price_diff', 'starrating_diff', 'count_window',
       'comp_avg_rate', 'comp_avg_inv', 'comp_avg_rate_percent_diff',
       'prop_starrating_mean', 'prop_starrating_median',
       'prop_review_score_mean', 'prop_review_score_median',
       'prop_location_score2_mean', 'prop_location_score2_median',
       'prop_brand_bool_mean', 'prop_brand_bool_median',
       'prop_log_historical_price_mean

In [11]:
output = model.predict(X)

In [12]:
print(len(y))
print(len(output))
y_grouped = []
output_grouped = []
start = 0
for cnt in qids:
    end = start + cnt
    y_grouped.append(y[start:end])
    output_grouped.append(output[start:end])
    start = end

3966677
3966677


In [13]:
print(len(y_grouped))
print(len(output_grouped))

159784
159784


In [14]:
ndcg_results = []
for i in range(len(y_grouped)):
    ndcg = ndcg_score([y_grouped[i]], [output_grouped[i]], k=5)
    ndcg_results.append(ndcg)
    if i % 10000 == 0:
        print(f'Group {i}, ndcg:{ndcg}.')
# print(ndcg_results)
print(np.mean(ndcg_results))

Group 0, ndcg:0.0.
Group 10000, ndcg:0.0.
Group 20000, ndcg:0.6309297535714573.
Group 30000, ndcg:0.6309297535714573.
Group 40000, ndcg:0.4306765580733929.
Group 50000, ndcg:0.0.
Group 60000, ndcg:0.6309297535714573.
Group 70000, ndcg:1.0.
Group 80000, ndcg:0.0.
Group 90000, ndcg:0.1508084140781908.
Group 100000, ndcg:0.0.
Group 110000, ndcg:0.6309297535714573.
Group 120000, ndcg:0.0.
Group 130000, ndcg:0.38685280723454163.
Group 140000, ndcg:0.0.
Group 150000, ndcg:0.3500439500444236.
0.4809896915197483


In [10]:
joblib.dump(model, 'models/lightgbm.pkl')

['models/lightgbm.pkl']